### GAN Attempt!

In [ ]:
!pip install transformers
!pip install datasets

In [ ]:
"""
Setup
"""

import torch
import torch.nn as nn
import torch.optim as optim
from transformers import (
    AutoModelForSeq2SeqLM,
    AutoTokenizer,
)
import pandas as pd
from datasets import Dataset

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

%cd drive/MyDrive/CS\ 224N/CS\ 224N\ Project
%ls # verify that you are in the right directory

In [ ]:
"""
Define the generator (use the pre-trained BART implementation)
"""

# bart-base checkpoint pre-trained on our dataset
# (can also try generically pre-trained bart base)
model_dir = 'bart-base-checkpoint-204000'
tokenizer = AutoTokenizer.from_pretrained(model_dir)
netG = AutoModelForSeq2SeqLM.from_pretrained(model_dir)
print(netG)

BartForConditionalGeneration(
  (model): BartModel(
    (shared): Embedding(50265, 768, padding_idx=1)
    (encoder): BartEncoder(
      (embed_tokens): Embedding(50265, 768, padding_idx=1)
      (embed_positions): BartLearnedPositionalEmbedding(1026, 768)
      (layers): ModuleList(
        (0): BartEncoderLayer(
          (self_attn): BartAttention(
            (k_proj): Linear(in_features=768, out_features=768, bias=True)
            (v_proj): Linear(in_features=768, out_features=768, bias=True)
            (q_proj): Linear(in_features=768, out_features=768, bias=True)
            (out_proj): Linear(in_features=768, out_features=768, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=768, out_features=3072, bias=True)
          (fc2): Linear(in_features=3072, out_features=768, bias=True)
          (final_layer_norm): LayerNorm((768,), eps=1e-05,

In [ ]:
"""
Define convolutional discriminator
"""

nc = 1
ndf = 64

class Discriminator(nn.Module):
    def __init__(self, ngpu):
        super(Discriminator, self).__init__()
        self.ngpu = ngpu
        self.main = nn.Sequential(
            #Reshaping input
            nn.Upsample(size=(64, 64)), #bring image from 1, 1, 1, 64 --> 1, 1, 64, 64
            # input is (nc) x 64 x 64 | our input is 1 x 64
            nn.Conv2d(nc, ndf, 4, 2, 1, bias=False),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf) x 32 x 32
            nn.Conv2d(ndf, ndf * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 2),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf*2) x 16 x 16
            nn.Conv2d(ndf * 2, ndf * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 4),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf*4) x 8 x 8
            nn.Conv2d(ndf * 4, ndf * 8, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 8),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf*8) x 4 x 4
            nn.Conv2d(ndf * 8, 1, 4, 1, 0, bias=False),
            nn.Sigmoid()
        )

    def forward(self, input):
        #print(len(input.shape))
        return self.main(input)

In [ ]:
ngpu = 1
netD = Discriminator(ngpu).to(device)

In [ ]:
from transformers import pipeline
sentiment = pipeline(model='finiteautomata/bertweet-base-sentiment-analysis')

In [ ]:
def get_sentiment_tokens(comment):
  max_input_length = 128
  if len(comment) > 128:
    comment = comment[:128]
  comment_sentiment = sentiment([comment])[0]
  comment_sentiment = comment_sentiment['label'] + ': ' + str(comment_sentiment['score'])
  comment_sentiment_tokens = tokenizer(comment_sentiment, max_length=128, padding='max_length', truncation=True, return_tensors="pt")
  return comment_sentiment_tokens['input_ids'].tolist()

In [ ]:
"""
Loss functions and optimizers
"""
# Size of generator input
nz = 512
# Optim params
lr = 0.0002
beta1 = 0.5

# Initialize BCELoss function
criterion = nn.BCELoss()

# Create batch of latent vectors that we will use to visualize
#  the progression of the generator
fixed_noise = torch.randn(64, nz, 1, 1, device=device)

# Establish convention for real and fake labels during training
real_label = 1.
fake_label = 0.

# Setup Adam optimizers for both G and D
optimizerD = optim.Adam(netD.parameters(), lr=lr, betas=(beta1, 0.999))
optimizerG = optim.Adam(netG.parameters(), lr=lr, betas=(beta1, 0.999))

In [ ]:
train_df = pd.read_csv('aita_train_set.csv')[['text', 'comments']]
valid_df = pd.read_csv('aita_valid_set.csv')[['text', 'comments']]
test_df = pd.read_csv('aita_test_set.csv')[['text', 'comments']]

In [ ]:
train_data_txt = Dataset.from_pandas(train_df)
validation_data_txt = Dataset.from_pandas(valid_df)
test_data_txt = Dataset.from_pandas(test_df)
print(train_data_txt)
print(validation_data_txt)
print(test_data_txt)

Dataset({
    features: ['text', 'comments'],
    num_rows: 81614
})
Dataset({
    features: ['text', 'comments'],
    num_rows: 998
})
Dataset({
    features: ['text', 'comments'],
    num_rows: 998
})


In [ ]:
"""
Preprocess
"""

encoder_max_length = 256  # changed from 256
decoder_max_length = 64  # changed from 64

def batch_tokenize_preprocess(batch, tokenizer, max_source_length, max_target_length):
    source, target = batch["text"], batch["comments"]
    source_tokenized = tokenizer(
        source, padding="max_length", truncation=True, max_length=max_source_length, return_tensors="pt"
    )
    target_tokenized = tokenizer(
        target, padding="max_length", truncation=True, max_length=max_target_length, return_tensors="pt"
    )

    batch = {k: v for k, v in source_tokenized.items()}
    # Ignore padding in the loss
    batch["labels"] = [
        [-100 if token == tokenizer.pad_token_id else token for token in l]
        for l in target_tokenized["input_ids"]
    ]
    return batch


train_data = train_data_txt.map(
    lambda batch: batch_tokenize_preprocess(
        batch, tokenizer, encoder_max_length, decoder_max_length
    ),
    batched=True,
    remove_columns=train_data_txt.column_names,
)

validation_data = validation_data_txt.map(
    lambda batch: batch_tokenize_preprocess(
        batch, tokenizer, encoder_max_length, decoder_max_length
    ),
    batched=True,
    remove_columns=validation_data_txt.column_names,
)

Map:   0%|          | 0/81614 [00:00<?, ? examples/s]

Map:   0%|          | 0/998 [00:00<?, ? examples/s]

In [ ]:
print(train_data)

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 81614
})


In [ ]:
print(len(train_data[0]['input_ids']))

256


In [ ]:
fixed_validation_index = 17
fixed_validation_inputs = valid_df.iloc[fixed_validation_index]['text']
fixed_validation_data = tokenizer(fixed_validation_inputs, max_length=512, padding='max_length', truncation=True, return_tensors="pt")

In [ ]:
# Training Loop

# Lists to keep track of progress
img_list = []
G_losses = []
D_losses = []
iters = 0
num_epochs = 1
max_input_length = 512

print("Starting Training Loop...")
# For each epoch
for epoch in range(num_epochs):
    # todo: batch this/use a dataloader
    for i in range(len(validation_data)):
        data = validation_data[i]
        ############################
        # (1) Update D network: maximize log(D(x)) + log(1 - D(G(z)))
        ###########################
        ## Train with all-real batch
        netD.zero_grad()
        # Format batch
        #print(len(data['labels']))
        sentiment_tokens = get_sentiment_tokens(str(valid_df.iloc[i]['comments']))
        real_cpu = torch.tensor(data['labels'] + sentiment_tokens[0], dtype=torch.float32)
        #real_cpu = data['labels']
        #print(real_cpu.shape)
        # real_cpu = real_cpu.unsqueeze(0)
        # real_cpu = real_cpu.unsqueeze(0)
        # real_cpu = real_cpu.unsqueeze(0)
        real_cpu = real_cpu.view(1, 1, 1, 64 + 128) #these are the comment tokens
        #print(real_cpu.shape)
        real_cpu = real_cpu.to(device)
        b_size = real_cpu.size(0)
        label = torch.full((b_size,), real_label, dtype=torch.float, device=device)

        #discriminator will train off of true comments in the real batch pass
        # Forward pass real batch through D
        output = netD(real_cpu).view(-1) 
        # Calculate loss on all-real batch
        errD_real = criterion(output, label)
        # Calculate gradients for D in backward pass
        errD_real.backward()
        D_x = output.mean().item()

        ## Train with all-fake batch
        # Generate batch of latent vectors
        # noise = torch.randn(b_size, nz, 1, 1, device=device)
        # print(inputs['input_ids'].shape)
        # output = model.generate(**inputs, num_beams=8, do_sample=True, min_length=10, max_length=64)
        # Generate fake image batch with G

        inputs = valid_df.iloc[i]['text']
        data = tokenizer(inputs, max_length=max_input_length, padding='max_length', truncation=True, return_tensors="pt")
        fake = netG.generate(**data, num_beams=8, do_sample=True, min_length=10, max_length=64) #generate a fake comment
        decoded_fake_comment = tokenizer.batch_decode(fake, skip_special_tokens=True)
        sentiment_tokens = get_sentiment_tokens(decoded_fake_comment[0])
        inp_tensor = torch.tensor(sentiment_tokens[0], dtype=torch.long).unsqueeze(0)
        fake = torch.cat((inp_tensor, fake), dim=1)
        label.fill_(fake_label)
        # Classify all fake batch with D
        #print(fake.shape)
        fake = fake.type(torch.float32)
        fake = fake.view(1, 1, 1, -1)
        fake = fake.detach().to(device)
        output = netD(fake).view(-1)
        # Calculate D's loss on the all-fake batch
        errD_fake = criterion(output, label)
        # Calculate the gradients for this batch, accumulated (summed) with previous gradients
        errD_fake.backward()
        D_G_z1 = output.mean().item()
        # Compute error of D as sum over the fake and the real batches
        errD = errD_real + errD_fake
        # Update D
        optimizerD.step()

        ############################
        # (2) Update G network: maximize log(D(G(z)))
        ###########################
        netG.zero_grad()
        label.fill_(real_label)  # fake labels are real for generator cost
        # Since we just updated D, perform another forward pass of all-fake batch through D
        output = netD(fake).view(-1)
        # Calculate G's loss based on this output
        errG = criterion(output, label)
        # Calculate gradients for G
        errG.backward()
        D_G_z2 = output.mean().item()
        # Update G
        optimizerG.step()

        # Output training stats
        if i % 5 == 0:
            print('[%d/%d][%d/%d]\tLoss_D: %.4f\tLoss_G: %.4f\tD(x): %.4f\tD(G(z)): %.4f / %.4f'
                  % (epoch, num_epochs, i, len(validation_data),
                     errD.item(), errG.item(), D_x, D_G_z1, D_G_z2))

        # Save Losses for plotting later
        G_losses.append(errG.item())
        D_losses.append(errD.item())

        if iters == 5: netG.save_pretrained('pear/pear-save-initial')
        # Check how the generator is doing by saving G's output on fixed_noise
        if (iters != 0 and iters % 500 == 0) or ((epoch == num_epochs-1) and (i == len(train_data)-1)):
            netG.save_pretrained('pear/pear-save-halfway')
            with torch.no_grad():
                fake = netG.generate(**fixed_validation_data, num_beams=8, do_sample=True, min_length=10, max_length=64).detach()
            # img_list.append(vutils.make_grid(fake, padding=2, normalize=True))

        iters += 1

netG.save_pretrained('pear/pear-save-final')

Starting Training Loop...
[0/1][0/998]	Loss_D: 1.2201	Loss_G: 6.7044	D(x): 0.4763	D(G(z)): 0.3802 / 0.0012
[0/1][5/998]	Loss_D: 0.0771	Loss_G: 3.8508	D(x): 0.9462	D(G(z)): 0.0216 / 0.0213
[0/1][10/998]	Loss_D: 0.0014	Loss_G: 8.8193	D(x): 0.9988	D(G(z)): 0.0001 / 0.0001
[0/1][15/998]	Loss_D: 0.0009	Loss_G: 7.6370	D(x): 0.9997	D(G(z)): 0.0006 / 0.0005
[0/1][20/998]	Loss_D: 0.0005	Loss_G: 7.7316	D(x): 0.9999	D(G(z)): 0.0005 / 0.0004
[0/1][25/998]	Loss_D: 0.0039	Loss_G: 7.1750	D(x): 0.9969	D(G(z)): 0.0008 / 0.0008
[0/1][30/998]	Loss_D: 0.0012	Loss_G: 6.8266	D(x): 0.9999	D(G(z)): 0.0011 / 0.0011
[0/1][35/998]	Loss_D: 0.0025	Loss_G: 8.5001	D(x): 0.9977	D(G(z)): 0.0002 / 0.0002
[0/1][40/998]	Loss_D: 0.0064	Loss_G: 6.9582	D(x): 0.9947	D(G(z)): 0.0010 / 0.0010
[0/1][45/998]	Loss_D: 0.0008	Loss_G: 7.2637	D(x): 1.0000	D(G(z)): 0.0007 / 0.0007
[0/1][50/998]	Loss_D: 0.0047	Loss_G: 6.2717	D(x): 0.9974	D(G(z)): 0.0021 / 0.0019
[0/1][55/998]	Loss_D: 0.2576	Loss_G: 5.2311	D(x): 0.7737	D(G(z)): 0.0011 /